---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [19]:
import pandas as pd
import numpy as np

In [20]:
energy = pd.read_excel('Energy Indicators.xls',
                       skiprows = list(range(17)),
                       usecols = list(range(2,6)),
                       skipfooter = 38)
energy.head()

,Unnamed: 0,Petajoules,Gigajoules,%
0,Afghanistan,321,10,78.669280
1,Albania,102,35,100.000000
2,Algeria,1959,51,0.551010
3,American Samoa,...,...,0.641026
4,Andorra,9,121,88.695650


In [21]:
header = {'Unnamed: 0': 'Country',
          'Petajoules':'Energy Supply',
          'Gigajoules':'Energy Supply per Capita',
          '%':'% Renewable'}
energy.rename(columns = header, inplace = True)
energy.replace('...', np.NaN, inplace = True)
energy['Energy Supply'] *= 1000000

def remove_digit_and_paren(country_name):
    new_country_name = ''.join([ch for ch in country_name if not ch.isdigit()])
    i = new_country_name.find('(')
    if i > -1:
        return new_country_name[:i].strip()
    else:
        return new_country_name.strip()

energy['Country'] = energy['Country'].apply(remove_digit_and_paren)

simple_country_names = {"Republic of Korea"                                   : "South Korea",
                        "United States of America"                            : "United States",
                        "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                        "China, Hong Kong Special Administrative Region"      : "Hong Kong"}
energy.replace({"Country": simple_country_names}, inplace = True)


In [22]:
# Check country renaming is correct
energy.where((energy['Country'] == "Hong Kong")
             | (energy['Country'] == "South Korea")
             | (energy['Country'] == "United States")
             | (energy['Country'] == "United Kingdom")
             | (energy['Country'] == 'Bolivia')
             | (energy['Country'] == 'Finland')
             | (energy['Country'] == 'Switzerland')).dropna()

,Country,Energy Supply,Energy Supply per Capita,% Renewable
24,Bolivia,3.360000e+08,32.0,31.477120
43,Hong Kong,5.850000e+08,82.0,0.000000
72,Finland,1.374000e+09,252.0,19.536570
164,South Korea,1.100700e+10,221.0,2.279353
197,Switzerland,1.113000e+09,136.0,57.745480
214,United Kingdom,7.920000e+09,124.0,10.600470
216,United States,9.083800e+10,286.0,11.570980


In [23]:
energy.iloc[energy['Energy Supply per Capita'].idxmax()]

Country                      Iceland
Energy Supply               3.12e+08
Energy Supply per Capita         957
% Renewable                  99.9724
Name: 95, dtype: object

In [24]:
energy.iloc[energy['Energy Supply per Capita'].idxmin()]

Country                     South Sudan
Energy Supply                   2.8e+07
Energy Supply per Capita              2
% Renewable                    0.423729
Name: 189, dtype: object

In [25]:
GDP = pd.read_csv('world_bank.csv',
                  skiprows = 4)
standard_country_names = {"Korea, Rep."         : "South Korea", 
                          "Iran, Islamic Rep."  : "Iran",
                          "Hong Kong SAR, China": "Hong Kong"}
GDP.replace({"Country Name": standard_country_names}, inplace = True)
GDP.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,Unnamed: 62
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.498933e+09,2.467704e+09,2.584464e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,1.006667e+09,...,1.248694e+10,1.593680e+10,1.793024e+10,2.053654e+10,2.026425e+10,2.061610e+10,1.921556e+10,1.946902e+10,2.081530e+10,NaN
2,Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,7.549239e+10,8.252614e+10,1.041158e+11,1.139232e+11,1.249125e+11,1.267302e+11,1.026212e+11,9.533720e+10,1.242094e+11,NaN
3,Albania,ALB,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.204421e+10,1.192695e+10,1.289087e+10,1.231978e+10,1.277628e+10,1.322824e+10,1.138693e+10,1.188368e+10,1.303935e+10,NaN
4,Andorra,AND,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3.660531e+09,3.355695e+09,3.442063e+09,3.164615e+09,3.281585e+09,3.350736e+09,2.811489e+09,2.877312e+09,3.012914e+09,NaN


In [26]:
GDP.columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', 'Unnamed: 62'],
      dtype='object')

In [27]:
# Check country renaming is correct
GDP.where((GDP['Country Name'] == "Hong Kong")
          | (GDP['Country Name'] == "South Korea")
          | (GDP['Country Name'] == "United States")
          | (GDP['Country Name'] == "United Kingdom")
          | (GDP['Country Name'] == 'Bolivia')
          | (GDP['Country Name'] == 'Finland')
          | (GDP['Country Name'] == 'Switzerland')
          | (GDP['Country Name'] == 'Samoa')
         ).dropna(how = 'all')

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,Unnamed: 62
26,Bolivia,BOL,GDP (current US$),NY.GDP.MKTP.CD,5.631101e+08,6.125189e+08,6.697225e+08,7.211430e+08,8.125431e+08,9.088745e+08,...,1.733999e+10,1.964963e+10,2.396303e+10,2.708450e+10,3.065934e+10,3.299619e+10,3.300020e+10,3.394113e+10,3.750864e+10,NaN
35,Switzerland,CHE,GDP (current US$),NY.GDP.MKTP.CD,9.522747e+09,1.071271e+10,1.187998e+10,1.306364e+10,1.448056e+10,1.534674e+10,...,5.415065e+11,5.837830e+11,6.995796e+11,6.680436e+11,6.885042e+11,7.091826e+11,6.792892e+11,6.687453e+11,6.788873e+11,NaN
73,Finland,FIN,GDP (current US$),NY.GDP.MKTP.CD,5.224102e+09,5.921659e+09,6.340581e+09,6.885920e+09,7.766655e+09,8.589340e+09,...,2.514990e+11,2.477998e+11,2.736742e+11,2.567065e+11,2.699801e+11,2.726093e+11,2.324648e+11,2.386777e+11,2.518849e+11,NaN
79,United Kingdom,GBR,GDP (current US$),NY.GDP.MKTP.CD,7.232805e+10,7.669436e+10,8.060194e+10,8.544377e+10,9.338760e+10,1.005958e+11,...,2.382826e+12,2.441173e+12,2.619700e+12,2.662085e+12,2.739819e+12,3.022828e+12,2.885570e+12,2.650850e+12,2.622434e+12,NaN
94,Hong Kong,HKG,GDP (current US$),NY.GDP.MKTP.CD,1.320797e+09,1.383682e+09,1.612346e+09,1.935298e+09,2.206466e+09,2.435079e+09,...,2.140464e+11,2.286377e+11,2.485136e+11,2.626294e+11,2.756969e+11,2.914594e+11,3.093836e+11,3.208812e+11,3.414493e+11,NaN
124,South Korea,KOR,GDP (current US$),NY.GDP.MKTP.CD,3.957874e+09,2.417238e+09,2.813934e+09,3.988246e+09,3.458518e+09,3.120308e+09,...,9.019350e+11,1.094499e+12,1.202464e+12,1.222807e+12,1.305605e+12,1.411334e+12,1.382764e+12,1.414804e+12,1.530751e+12,NaN
249,United States,USA,GDP (current US$),NY.GDP.MKTP.CD,5.433000e+11,5.633000e+11,6.051000e+11,6.386000e+11,6.858000e+11,7.437000e+11,...,1.441874e+13,1.496437e+13,1.551793e+13,1.615526e+13,1.669152e+13,1.742761e+13,1.812071e+13,1.862448e+13,1.939060e+13,NaN
258,Samoa,WSM,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,5.609595e+08,6.430467e+08,7.397851e+08,8.011686e+08,8.048085e+08,8.035895e+08,8.039858e+08,7.863563e+08,8.566265e+08,NaN


In [28]:
ScimEn = pd.read_excel('scimagojr-3.xlsx')
ScimEn.head()

,Rank,Country,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
0,1,China,167992,167369,1057626,722578,6.30,176
1,2,United States,126158,123332,1296212,444998,10.27,278
2,3,Japan,37948,37612,316956,85620,8.35,155
3,4,United Kingdom,28998,28176,335914,64609,11.58,170
4,5,India,24872,24300,215787,68150,8.68,141


In [29]:
df = pd.merge(GDP, pd.merge(ScimEn, energy, on='Country'), left_on = 'Country Name', right_on = 'Country')
df.set_index('Country',inplace=True)
df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable
Country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,1.006667e+09,...,142,10,10,9,2,0.90,2,3.210000e+08,10.0,78.66928
Angola,Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,106,62,62,196,3,3.16,8,6.420000e+08,27.0,70.90909
Albania,Albania,ALB,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,116,38,38,246,17,6.47,8,1.020000e+08,35.0,100.00000
Andorra,Andorra,AND,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,177,2,2,18,0,9.00,1,9.000000e+06,121.0,88.69565
United Arab Emirates,United Arab Emirates,ARE,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,41,1715,1689,12744,1424,7.43,49,2.710000e+09,300.0,0.00000


In [30]:
df = df[['Rank',
         'Documents',
         'Citable documents',
         'Citations',
         'Self-citations',
         'Citations per document',
         'H index',
         'Energy Supply', 
         'Energy Supply per Capita',
         '% Renewable',
         '2009',
         '2010',
         '2011',
         '2012',
         '2013',
         '2014',
         '2015',
         '2016',
         '2017'
        ]]
df.head()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2009,2010,2011,2012,2013,2014,2015,2016,2017
Country,,,,,,,,,,,,,,,,,,,
Afghanistan,142,10,10,9,2,0.90,2,3.210000e+08,10.0,78.66928,1.248694e+10,1.593680e+10,1.793024e+10,2.053654e+10,2.026425e+10,2.061610e+10,1.921556e+10,1.946902e+10,2.081530e+10
Angola,106,62,62,196,3,3.16,8,6.420000e+08,27.0,70.90909,7.549239e+10,8.252614e+10,1.041158e+11,1.139232e+11,1.249125e+11,1.267302e+11,1.026212e+11,9.533720e+10,1.242094e+11
Albania,116,38,38,246,17,6.47,8,1.020000e+08,35.0,100.00000,1.204421e+10,1.192695e+10,1.289087e+10,1.231978e+10,1.277628e+10,1.322824e+10,1.138693e+10,1.188368e+10,1.303935e+10
Andorra,177,2,2,18,0,9.00,1,9.000000e+06,121.0,88.69565,3.660531e+09,3.355695e+09,3.442063e+09,3.164615e+09,3.281585e+09,3.350736e+09,2.811489e+09,2.877312e+09,3.012914e+09
United Arab Emirates,41,1715,1689,12744,1424,7.43,49,2.710000e+09,300.0,0.00000,2.535474e+11,2.897873e+11,3.506660e+11,3.745906e+11,3.901076e+11,4.031371e+11,3.581351e+11,3.570451e+11,3.825751e+11


In [31]:
def answer_one():
    return df.loc[df['Rank'].isin(list(range(16)))]

answer_one()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2009,2010,2011,2012,2013,2014,2015,2016,2017
Country,,,,,,,,,,,,,,,,,,,
Australia,14,12635,12397,157471,27620,12.46,131,5.386000e+09,231.0,11.810810,9.264482e+11,1.144261e+12,1.394281e+12,1.543411e+12,1.573697e+12,1.464955e+12,1.349034e+12,1.208039e+12,1.323421e+12
Brazil,13,12705,12559,99136,24398,7.80,103,1.214900e+10,59.0,69.648030,1.667020e+12,2.208872e+12,2.616202e+12,2.465189e+12,2.472807e+12,2.455994e+12,1.802214e+12,1.793989e+12,2.055506e+12
Canada,8,22908,22465,332093,62436,14.50,177,1.043100e+10,296.0,61.945430,1.371153e+12,1.613464e+12,1.788648e+12,1.824289e+12,1.842628e+12,1.799269e+12,1.559623e+12,1.535768e+12,1.653043e+12
China,1,167992,167369,1057626,722578,6.30,176,1.271910e+11,93.0,19.754910,5.109954e+12,6.100620e+12,7.572554e+12,8.560547e+12,9.607224e+12,1.048237e+13,1.106467e+13,1.119099e+13,1.223770e+13
Germany,6,24407,23963,231800,48820,9.50,151,1.326100e+10,165.0,17.901530,3.418005e+12,3.417095e+12,3.757698e+12,3.543984e+12,3.752514e+12,3.890607e+12,3.375611e+12,3.477796e+12,3.677439e+12
Spain,15,12478,12270,195663,36353,15.68,138,4.923000e+09,106.0,37.968590,1.499100e+12,1.431617e+12,1.488067e+12,1.336019e+12,1.361854e+12,1.376911e+12,1.197790e+12,1.237255e+12,1.311320e+12
France,9,17569,17230,203486,43210,11.58,139,1.059700e+10,166.0,17.020280,2.690222e+12,2.642610e+12,2.861408e+12,2.683825e+12,2.811078e+12,2.852166e+12,2.438208e+12,2.465134e+12,2.582501e+12
United Kingdom,4,28998,28176,335914,64609,11.58,170,7.920000e+09,124.0,10.600470,2.382826e+12,2.441173e+12,2.619700e+12,2.662085e+12,2.739819e+12,3.022828e+12,2.885570e+12,2.650850e+12,2.622434e+12
India,5,24872,24300,215787,68150,8.68,141,3.319500e+10,26.0,14.969080,1.323940e+12,1.656617e+12,1.823050e+12,1.827638e+12,1.856722e+12,2.039127e+12,2.102391e+12,2.274230e+12,2.597491e+12


### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [32]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [33]:
def answer_two():
    return df.count().max() - 15

answer_two()

147

## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [49]:
def answer_three():
    Top15 = answer_one()
    years = ['20' + '{:02}'.format(i) for i in range(9, 18)]
    return (Top15[years]
                .mean(axis=1)
                .sort_values(ascending=False)
                .rename('avgGDP'))

answer_three()

Country
United States         1.681236e+13
China                 9.102959e+12
Japan                 5.279408e+12
Germany               3.590083e+12
United Kingdom        2.669698e+12
France                2.669684e+12
Brazil                2.170866e+12
Italy                 2.063179e+12
India                 1.944578e+12
Russian Federation    1.733436e+12
Canada                1.665321e+12
Spain                 1.359993e+12
Australia             1.325283e+12
South Korea           1.274107e+12
Iran                  4.699707e+11
Name: avgGDP, dtype: float64

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [58]:
def answer_four():
    Top15 = answer_one()
    Top15['avgGDP'] = answer_three()
    Top15.sort_values(by='avgGDP', inplace=True, ascending=False)
    return Top15.iloc[5]['2017']-Top15.iloc[5]['2009']

answer_four()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


-107720976751.3501

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [59]:
def answer_five():
    Top15 = answer_one()
    return Top15['Energy Supply per Capita'].mean()

answer_five()

157.6

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [ ]:
def answer_six():
    Top15 = answer_one()
    return "ANSWER"

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [ ]:
def answer_seven():
    Top15 = answer_one()
    return "ANSWER"

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [ ]:
def answer_eight():
    Top15 = answer_one()
    return "ANSWER"

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [ ]:
def answer_nine():
    Top15 = answer_one()
    return "ANSWER"

In [ ]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [ ]:
#plot9() # Be sure to comment out plot9() before submitting the assignment!

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [ ]:
def answer_ten():
    Top15 = answer_one()
    return "ANSWER"

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [ ]:
def answer_eleven():
    Top15 = answer_one()
    return "ANSWER"

### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [ ]:
def answer_twelve():
    Top15 = answer_one()
    return "ANSWER"

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [ ]:
def answer_thirteen():
    Top15 = answer_one()
    return "ANSWER"

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [ ]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [ ]:
#plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!